# 5. 네이버 영화 리뷰

### requests와 BeautifulSoup으로 웹 크롤링을 하여 네이버 영화 리뷰 등록 날짜, 별점, 내용을 수집 

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import sys
import pandas as pd
from tqdm import tqdm_notebook
from tqdm import tqdm
import multiprocessing

In [2]:
tqdm.monitor_interval = 0
tqdm_notebook.monitor_interval = 0

In [3]:
movie_info_path='/home/ubuntu/miniii_su/movie/'

In [7]:
af='after'
be='before'
lis = pd.read_csv(movie_info_path+'data/naver_movie_all.csv')
movie_lis =[]

for i in range(len(lis['naver_movieCd'])):
    movie_lis.insert(i,str(lis['naver_movieCd'][i])[:-2]) # 네이버 moviecd 를 movie_lis에 삽입

def func12(code):
    try:
        lists = []
        test_url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='+code+'&type='+af+'&onlyActualPointYn=N&order=newest'
        resp = requests.get(test_url)
        html = BeautifulSoup(resp.content, 'html.parser')
        result = html.find('div', {'class':'score_total'}).find('strong').findChildren('em')[1].getText()
        total_count = int(result.replace(',', ''))
        print(code)

        for i in tqdm_notebook(range(1,int(total_count / 10) + 2)):
            try:
                url = test_url + '&page=' + str(i)
                resp = requests.get(url)
                html = BeautifulSoup(resp.content, 'html.parser')
                score_result = html.find('div', {'class': 'score_result'})
                lis = score_result.findAll('li')

                try:
                    for li in lis:                
                        a = []
                        created_at = li.find('dt').findAll('em')[-1].getText()[0:10]
                        created_at = created_at.replace(".","")
                        score = li.find('em').getText()
                        review_text = li.find('p').getText()
                        a = [created_at, score, review_text]
                        lists.append(a)

                except AttributeError:
                    pass

            except AttributeError:
                pass
            
    except AttributeError:
        pass
    
    df = pd.DataFrame(lists)
    df.to_csv(movie_info_path+'raw_data/'+af+'/'+code+'.csv',index=False)

# pool = multiprocessing.Pool(processes=8)
# pool.map(func12,movie_lis)
# pool.close()
# pool.join()

In [ ]:
pool = multiprocessing.Pool(processes=4)
pool.map(func12,movie_lis)
pool.close()
pool.join()